In [60]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go

In [61]:
np.random.seed(1)
X = np.random.uniform(-5, 5, 100)
epsilon = np.random.normal(0, 0.5, 100)
y = 3 * X**3 - 2 * X**2 + 5 + epsilon

In [62]:
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

In [63]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
linear_model = LinearRegressionModel()

In [64]:
class PolynomialRegressionModel(nn.Module):
    def __init__(self):
        super(PolynomialRegressionModel, self).__init__()
        self.poly = nn.Linear(3, 1)

    def forward(self, x):
        x_poly = torch.cat([x**3, x**2, x], dim=1)
        return self.poly(x_poly)

poly_model = PolynomialRegressionModel()

In [65]:
def train_model(model, X_tensor, y_tensor, num_epochs=1000, learning_rate=0.0001):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    loss_values = []

    for _ in range(num_epochs):
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())

    return loss_values

In [66]:
linear_loss = train_model(linear_model, X_tensor, y_tensor)
poly_loss = train_model(poly_model, X_tensor, y_tensor)

In [67]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(linear_loss)), y=linear_loss, mode='lines', name='Linear Regression'))
fig.add_trace(go.Scatter(x=np.arange(len(poly_loss)), y=poly_loss, mode='lines', name='Polynomial Regression (degree 3)'))

fig.update_layout(
    title="Training Loss for Linear and Polynomial Regression Models",
    xaxis=dict(title="Epochs"),
    yaxis=dict(title="Loss (MSE)"),
    showlegend=True
)

fig.show(config=dict(scrollZoom=True))

print("Linear Regression Model Parameters:")
for param in linear_model.parameters():
    print(param)

print("\nPolynomial Regression Model Parameters:")
for param in poly_model.parameters():
    print(param)

Linear Regression Model Parameters:
Parameter containing:
tensor([[38.5838]], requires_grad=True)
Parameter containing:
tensor([-3.8837], requires_grad=True)

Polynomial Regression Model Parameters:
Parameter containing:
tensor([[ 2.9704, -1.6564,  0.5682]], requires_grad=True)
Parameter containing:
tensor([-0.1969], requires_grad=True)


In [68]:
X_space = torch.arange(-5, 5, 0.001, dtype=torch.float32).view(-1, 1)

predicted_linear = linear_model(X_space).detach().numpy()
predicted_poly = poly_model(X_space).detach().numpy()

scatter = go.Scatter(x=X, y=y, mode='markers', name='True data')
linear_line = go.Scatter(x=X_space.numpy().flatten(), y=predicted_linear.flatten(), mode='lines', name='Linear model', line=dict(color='green'))
poly_line = go.Scatter(x=X_space.numpy().flatten(), y=predicted_poly.flatten(), mode='lines', name='Poly model', line=dict(color='red'))

layout = go.Layout(
    title="Linear and Poly Regression Fit Compare",
    xaxis=dict(title="X"),
    yaxis=dict(title="y"),
    showlegend=True
)

fig = go.Figure(data=[scatter, linear_line, poly_line], layout=layout)
fig.show(config=dict(scrollZoom=True))
